## 1. Load Gold Table

In [0]:
df_gold = spark.read.format("delta").load(
    "dbfs:/Volumes/workspace/credit-risk/credit-risk/gold"
)

df_gold.display()


## 2. Load Best Model From UC Volume

In [0]:
import mlflow
import mlflow.spark

best_model_path = "/Volumes/workspace/credit-risk/credit-risk/models/best_model"

best_model = mlflow.spark.load_model(
    best_model_path,
    dfs_tmpdir="dbfs:/Volumes/workspace/credit-risk/credit-risk/mlflow_tmp"
)

print("Model loaded successfully!")
best_model


## 3. Apply model to Gold Data (Scoring)

In [0]:
df_scored = best_model.transform(df_gold)

df_scored.select(
    "risk",
    "prediction",
    "probability"
).show(20, truncate=False)


## 4. Save scored output to Delta (Scoring Layer)

In [0]:
output_path = "dbfs:/Volumes/workspace/credit-risk/credit-risk/scoring"

df_scored.write.format("delta").mode("overwrite").save(output_path)

print(f"Scoring table saved to: {output_path}")



## 5. Load and validate scoring table

In [0]:
df_scoring = spark.read.format("delta").load(output_path)

df_scoring.display()


## 6. List directory to confirm objects

In [0]:
display(dbutils.fs.ls("dbfs:/Volumes/workspace/credit-risk/credit-risk/"))

